In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Brownian Bridge Simulation and Metric Estimation on Lie Groups and Homogeneous Spaces: GL3 and SPD3

Mathias Højgaard Jensen, Stefan Sommer and Sarang Joshi

In [ ]:
%cd ..
from src.groups.GLN import *
G = GLN(3)
print(G)

from src.manifolds.SPDN import *
M = SPDN(3)
print(M)

from src.plotting import *
figsize = 12,12
plt.rcParams['figure.figsize'] = figsize

In [ ]:
from src.group import invariant_metric
invariant_metric.initialize(G)

from src.group import energy
energy.initialize(G)

In [ ]:
v=0.*np.array([.5,0,0,0,0,0,0,0,0])+1e-6*np.random.normal(size=G.dim.eval()) # must be non-singular for Expm derivative
xiv=G.VtoLAf(v)
g = G.expf(xiv)
print(g)

x = np.eye((M.N.eval())).flatten()

In [ ]:
# quotient space
from src.group.quotient import *
horz_vert_splitf = G.function(lambda y: horz_vert_split(y,lambda g: M.act(g,x),G,M)) # horizontal, vertical splitting
(_,_,proj_horz,proj_ns,horz)=horz_vert_splitf(g)
print(proj_horz,proj_ns,horz)

In [ ]:
# sample data
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

G.sigma.set_value(np.sqrt(.1)*np.eye(G.emb_dim.eval())) # set metric
K = 1024
obss = np.zeros((K,)+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    obss[i] = gs[-1]

# plot samples
newfig()
for i in range(K):
    G.plotg(obss[i])
plt.show()

# on SPD(3)
obss_M = np.zeros((K,M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    obss_M[i] = M.actf(obss[i],x)
    M.plotx(obss_M[i])
plt.show()

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# parameters
g0 = G.sym_element()
thetas = (g0, G.sigma,)
thetas_true = [g]+[theta.eval() for theta in thetas[1:]]

# guide function
# phi = lambda g,v: T.tensordot(G.inv(G.sigma),G.LAtoV(G.invtrns(G.inv(g),v)-G.e),(1,0))
phi = lambda g,v: T.tensordot(G.inv(G.sigma),G.LAtoV(G.log(G.invtrns(G.inv(g),v))),(1,0))

# (Brownian_inv_guided,Brownian_inv_guidedf) = get_guided_likelihood(G,G.sde_Brownian_inv,phi,lambda g: G.sigma, g0, thetas, A=G.gG, integration='stratonovich')
(Brownian_inv_guided,Brownian_inv_guidedf) = get_guided_likelihood(
    G,G.sde_Brownian_inv,phi,lambda g: G.sigma, 
    A=G.gG, integration='stratonovich')

w = G.psif(v)
(ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,w,dWsf(G.dim.eval()))[:4]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

newfig()
G.plot_path(gs)
G.plotg(w,color='k')
# plt.savefig('GL3-bridge.pdf')
plt.show()

# on SPD(3)
newfig()
M.plot()
M.plot_path(M.actsf(gs,x))
M.plotx(M.actf(w,x),color='k')
# plt.savefig('SPD3-bridge.pdf')
plt.show()

In [ ]:
# fiber Brownian motion
from src.group.quotient import *
sde_Brownian_inv_fiber = get_sde_fiber(G.sde_Brownian_inv,lambda g: M.act(g,x),G,M)
Brownian_inv_fiber = lambda g,dWt: integrate_sde(sde_Brownian_inv_fiber,
                                                  integrator_stratonovich,
                                                  None,
                                                 g,None,dWt)
Brownian_inv_fiberf = theano.function([g0,dWt], Brownian_inv_fiber(g0,dWt))

# make L simulations
L = 3
gsl = np.zeros((L,)+g.shape)
vl = G.psif(v)
for l in range(L):
    (ts,gs) = Brownian_inv_fiberf(vl,dWsf(G.dim.eval()))[:2]
    gsl[l] = gs[-1]
    vl = gs[-1]
newfig()
step = 5
for l in range(0,L,step):
    (ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,gsl[l],dWsf(G.dim.eval()))[:4]
    G.plot_path(gs)
    G.plotg(gsl[l],color='k')
# plt.savefig('GL3-bridges_fiber.pdf')
plt.show()

# on SPD(3)
newfig()
M.plot()
for l in range(0,L,step):
    (ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,gsl[l],dWsf(G.dim.eval()))[:4]
    M.plot_path(M.actsf(gs,x))
    M.plotx(M.actf(gsl[l],x),color='k')
#     plt.savefig('SPD3-bridges_fiber.pdf')
plt.show()

In [ ]:
options = {}
options['samples_per_obs'] = 2
options['epochs'] = 30
options['learning_rate'] = 1
options['varphi_update_rate'] = 1.
options['verbose'] = True
options['initial'] = [g, #obss[0], # random value
                      .1*np.eye(G.dim.eval()),]
# options['update_v'] = lambda g: theano.gradient.disconnected_grad(Brownian_inv_fiber(g,dWs(G.dim))[1][-1])
# options['update_vf'] = lambda g: Brownian_inv_fiberf(g,dWsf(G.dim.eval()))[1][-1]

In [ ]:
# Transition density
v0 = G.sym_element()
p_Tf = theano.function([g0,v0],p_T(g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv))
log_p_Tf = theano.function([g0,v0],log_p_T(g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv))
dlog_p_Tf = theano.function([g0,v0],dlog_p_T(thetas,g0,v0,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv))

print(p_Tf(g,G.psif(v))) 
print(log_p_Tf(g,G.psif(v))) 
print(dlog_p_Tf(g,G.psif(v)))

In [ ]:
%%time
print(p_Tf(g,G.psif(v))) 

In [ ]:
%%time
print(dlog_p_Tf(g,G.psif(v)))

In [ ]:
%%time
L = options['samples_per_obs']
gsl = np.zeros((L,)+g.shape)
vl = G.psif(v)
for l in range(L):
#     (ts,gs) = Brownian_inv_fiberf(vl,dWsf(G.dim.eval()))
#     (ts,gs) = G.Brownian_invf(vl,dWsf(G.dim.eval()))
    (ts,gs,_,_) = Brownian_inv_guidedf(vl,w,dWsf(G.dim.eval()))[:4]
    gsl[l] = gs[-1]
    vl = gs[-1]

In [ ]:
n_steps.set_value(1)
print(n_steps.eval())

In [ ]:
# samples for MLE
G.sigma.set_value(.2*np.eye(G.dim.eval())) # set metric, uniform
# G.sigma.set_value(np.diag((.2,.2,1.5))) # anisotropic - this value is from SO(3), find a reasonable value for GL(3)

thetas_true = [g,G.sigma.eval()]

K = 64

obss = np.zeros((K,)+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    obss[i] = gs[-1]

# plot samples
newfig()
for i in range(K):
    G.plotg(obss[i])
plt.show()

# on SPD(3)
obss_M = np.zeros((K,M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    obss_M[i] = M.actf(obss[i],x)
    M.plotx(obss_M[i],color='k')
plt.show()

In [ ]:
# sample bridges
def lbridge_sampling(thetas,*args,**kwargs):
    g = thetas[0]
    G.sigma.set_value(thetas[1])
    return partial(bridge_sampling,g,Brownian_inv_guidedf,lambda: dWsf(G.dim.eval()),options)(*args,**kwargs)

log_phis = np.zeros((K,))
try:
    mpu.openPool()
    sol = mpu.pool.imap(partial(lbridge_sampling,options['initial']),mpu.inputArgs(obss,np.random.randint(1000,size=K)))
    res = list(sol)
    bridges = mpu.getRes(res,0)
    log_varphis = mpu.getRes(res,1)
    log_likelihoods = mpu.getRes(res,2)
except:
    mpu.closePool()
    raise
else:
    mpu.closePool()

# Plot on SPD(3)
newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
for i in range(bridges.shape[0]):
    M.plotx(M.actf(obss[i],x),color=colors[i])
    for j in range(bridges.shape[1]):
        gs = bridges[i,j]
        M.plot_path(M.actsf(gs,x),linewidth=.2,color=colors[i])
plt.show()

In [ ]:
%%time
# on GL(3)
from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (v,seed) = pars
    if seed:
        srng.seed(seed)
    g = thetas[0]
    G.sigma.set_value(thetas[1])
    return dlog_p_Tf(g,v)

def update_thetas(thetas, dthetas):
    g = thetas[0]
    sigma = thetas[1]
    
#     g = G.to_groupf(g+5.e-1*options['learning_rate']*dthetas[0])
    sigma += 2.e-3*options['learning_rate']*dthetas[1]
    sigma = .5*(sigma+sigma.T) # symmetrize
    # prevent negative eigenvalues in sigma
    (w,V) = np.linalg.eigh(sigma)
    sigma = np.dot(V,np.dot(np.diag(np.maximum(w,1e-3)),V.T))
    
    return (g,sigma)

# run MLE
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss,llog_p_T,update_thetas,options)

In [ ]:
# plot
plt.plot(range(options['epochs']),log_likelihoods)
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
plt.hlines(thetas_true[0].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
plt.plot(range(options['epochs']),thetass[1].reshape((thetass[1].shape[0],-1)))
plt.hlines(thetas_true[1].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
None

In [ ]:
%%time
# on SPD(3)
from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (v,seed) = pars
    if seed:
        srng.seed(seed)
    g = thetas[0]
    G.sigma.set_value(thetas[1])
    return dlog_p_Tf(g,lift_to_fiber(v,x,G,M)[0])

def update_thetas(thetas, dthetas):
    g = thetas[0]
    sigma = thetas[1]
    
#     g = G.to_groupf(g+5.e-1*options['learning_rate']*dthetas[0])
    _sigma = 2.5e-3*options['learning_rate']*dthetas[1]
    _sigma = .5*(_sigma+_sigma.T) # symmetrize    
    # projecting to the horizontal subspace of T_eG (orthogonal to the fiber)
    sigma += np.dot(proj_horz.T,np.dot(_sigma,proj_horz))
    
    # prevent negative eigenvalues in sigma
    (w,V) = scipy.linalg.eigh(sigma)
    sigma = np.dot(V,np.dot(np.diag(np.maximum(w,1e-3)),V.T))
    
    return (g,sigma)

# run MLE
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss_M,llog_p_T,update_thetas,options)

# plot
plt.plot(range(options['epochs']),log_likelihoods)
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
plt.hlines(thetas_true[0].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
plt.plot(range(options['epochs']),thetass[1].reshape((thetass[1].shape[0],-1)))
plt.hlines(thetas_true[1].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
None